In [1]:
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, experimental
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2

from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)


train_datagen = ImageDataGenerator(
    featurewise_center=True,
    validation_split = 0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2)

train_generator = train_datagen.flow_from_directory(
        './image_15000/train',
        target_size = (300,300),
        batch_size=128,
        #color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        './image_15000/test',
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=32,
#         color_mode ='rgb',
        shuffle =True,
        seed = 42,
        class_mode='categorical')

Found 13930 images belonging to 3 classes.
Found 3929 images belonging to 3 classes.


In [ ]:


model = Sequential()
model.add(Conv2D(8, kernel_size=(11,11),
                 activation='selu',strides= (2,2),
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(3,3)),)


model.add(Conv2D(32,(5,5), activation='relu',strides= (2,2)))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=3,
    epochs=5,
    validation_data=test_generator,
    validation_steps=3)

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 145, 145, 8)       2912      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        6432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 16)          4624      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 16)          2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 16)          0

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


# 정규화 X

In [ ]:
model = Sequential()
model.add(Conv2D(8, kernel_size=(11,11),
                 activation='selu',strides= (2,2),
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(3,3)),)
#experimental.preprocessing.Rescaling(1./255),

model.add(Conv2D(32,(5,5), activation='relu',strides= (2,2)))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=3,
    epochs=5,
    validation_data=test_generator,
    validation_steps=3)

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

In [ ]:
model = Sequential([
Input(shape=(300,300,3), name='input_layer'),
Conv2D(32,(3,3), activation='relu',name='Conv_layer1'),
BatchNormalization(),
MaxPooling2D(pool_size=(2,2),name='Pooling_layer2'),
Conv2D(32,(3,3), activation='relu',name='Conv_layer3'),
MaxPooling2D(pool_size=(2,2),name='Pooling_layer4'),
Conv2D(32,(3,3), activation='relu',name='Conv_layer5'),
MaxPooling2D(pool_size=(2,2),name='Pooling_layer6'),

Flatten(),
Dense(10, activation='relu',name='FC_layer7'),
Dense(3, activation='softmax', name='output_layer')
])

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=5,
    epochs=5,
    validation_data=test_generator,
    validation_steps=5)

model.save('model-3_정규화x')

test_loss, test_acc = model.evaluate(train_generator,  test_generator, verbose=2)
print(test_acc)